# SuperAI Season 4 - Level Individual Hackathon - Sleep Stages Classification <br> Classification

## Load data from database

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from tqdm.notebook import tqdm

In [ ]:
with open ('./database/arrays.npy' , 'rb') as file :

    arrays = np.load(file)

with open ('./database/labels.npy' , 'rb') as file :

    labels = np.load(file)

In [ ]:
print(arrays.shape , labels.shape)

(66607, 17280) (66607,)


In [ ]:
id2label = {
    0 : 'W' ,
    1 : 'N' ,
}
label2id = {
    'W' : 0 ,
    'N' : 1 ,
    'R' : 1
}

## Sleep Stages Dataset

In [ ]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 22.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , random_split
from torch.utils.data import Dataset
import torchmetrics
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

In [ ]:
class sleep_stages_dataset (Dataset) :

    def __init__ (self , train_path , label_path , transform = None , target_transform = None) :

        self.train_path = train_path
        self.label_path = label_path

        with open (train_path , 'rb') as file :

            self.train_arrays = np.load(file)

        with open (label_path , 'rb') as file :

            self.label_arrays = np.load(file)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) :

        return self.train_arrays.shape[0]

    def __getitem__ (self , idx) :

        data  = self.train_arrays[idx]
        label = self.label_arrays[idx]

        if self.transform :

            data = self.transform(data)

        if self.target_transform :

            label = self.target_transform(int(label))

        return data , label

In [ ]:
class OneHotEncodeTransform:

    def __init__(self, num_classes):

        self.num_classes = num_classes

    def __call__(self, target):

        return torch.nn.functional.one_hot(torch.tensor(target), num_classes=self.num_classes ).type(torch.float)

In [ ]:
transform = torch.Tensor
target_transform = OneHotEncodeTransform(num_classes = 3)

In [ ]:
train_path = './database/arrays.npy'
label_path = './database/labels.npy'

In [ ]:
train_dataset = sleep_stages_dataset(train_path = train_path , label_path = label_path , transform = transform , target_transform = target_transform)

In [ ]:
# Define the size of each split
train_size = int(0.8 * len(train_dataset))
validation_size = len(train_dataset) - train_size

# Split the dataset
train_dataset , validation_dataset = random_split(train_dataset, [train_size , validation_size])

train_loader = DataLoader(train_dataset , batch_size = 256 , shuffle = True , num_workers=11 )
validation_loader = DataLoader(validation_dataset , batch_size = 256 , shuffle = False , num_workers=11)

## ⚡ MaxVit - Torch Lightning ⚡

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class classifier(nn.Module):

    def __init__(self , num_features , num_targets):

        super().__init__()

        self.fc1 = nn.Linear(num_features, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.fc4 = nn.Linear(1024, 512)

        self.batch_norm1 = nn.BatchNorm1d(4096)
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.batch_norm4 = nn.BatchNorm1d(512)

        self.dropout = nn.Dropout(p = 0.25)
        self.output = nn.Linear(512, num_targets)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = F.relu(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.dropout(x)

        return F.softmax(self.output(x))

In [ ]:
class sleep_stages_classifier(L.LightningModule):

    def __init__(self , num_features , num_targets):

        super().__init__()

        self.classifier = classifier(num_features , num_targets)

        self.train_accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes = 3)
        self.val_accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes = 3)

    def forward (self , x) :

        return self.classifier(x)

    def training_step(self, batch, batch_idx):

        x , y = batch

        y_hat = self(x.to(device))

        loss = F.cross_entropy(y_hat.squeeze() , y.to(device))
        acc = self.train_accuracy(y_hat.squeeze().argmax(axis = 1), y.to(device).argmax(axis = 1))

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):

        x , y = batch

        y_hat = self(x)

        val_loss = F.cross_entropy(y_hat.squeeze(), y.to(device))
        val_acc = self.val_accuracy(y_hat.squeeze().argmax(axis = 1) , y.to(device).argmax(axis = 1))

        self.log('val_loss', val_loss, on_step = True, on_epoch = True, prog_bar = True)
        self.log('val_acc', val_acc, on_step = True, on_epoch = True, prog_bar = True)

    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(), lr = 1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.1)

        return [optimizer], [scheduler]

In [ ]:
sleep_stages_classifier_model = sleep_stages_classifier(num_features = 15360 , num_targets = 3)

In [ ]:
sleep_stages_classifier_model.to(device)

sleep_stages_classifier(
  (classifier): classifier(
    (fc1): Linear(in_features=17280, out_features=4096, bias=True)
    (fc2): Linear(in_features=4096, out_features=2048, bias=True)
    (fc3): Linear(in_features=2048, out_features=1024, bias=True)
    (fc4): Linear(in_features=1024, out_features=512, bias=True)
    (batch_norm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch_norm4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.25, inplace=False)
    (output): Linear(in_features=512, out_features=3, bias=True)
  )
  (train_accuracy): MulticlassAccuracy()
  (val_accuracy): MulticlassAccuracy()
)

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath='./models'  , filename='mlp_{epoch}-{val_acc:.2f}')

In [ ]:
trainer = L.Trainer(max_epochs = 100 , callbacks=[checkpoint_callback])
trainer.fit(model = sleep_stages_classifier_model, train_dataloaders = train_loader, val_dataloaders = validation_loader )

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more det

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
<ipython-input-16-2b4bf1d3edea>:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.output(x))
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
model = sleep_stages_classifier.load_from_checkpoint("/path/to/checkpoint.ckpt")
# disable randomness, dropout, etc...
model.eval()

# predict with the model
y_hat = model(x)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/checkpoint.ckpt'